In [3]:
import pandas as pd
import numpy as np
import re

import zipfile
import os

from google.colab import drive

In [2]:
#Get data zip-files
! wget #path_to_the_UKP_or_Amazon_zip_files

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [5]:
path_name = 'ukp' #'amazon'
for _, _, files in os.walk(path_name):
  for file in files:
    if file.endswith('.zip'):
      print(file)
      z = zipfile.ZipFile(path_name + '/' + file, 'r')
      z.extractall()

abortion.zip
cloning.zip
death_penalty.zip
gun_control.zip
marijuana_legalization.zip
minimum_wage.zip
nuclear_energy.zip
school_uniforms.zip


In [6]:
name_of_task = 'cloning'

In [7]:
merge_list = os.listdir(path="/content/" + name_of_task + "_merge")
merge_list

['854_cloning.txt.merge',
 '891_cloning.txt.merge',
 '987_cloning.txt.merge',
 '2970_cloning.txt.merge',
 '39_cloning.txt.merge',
 '86_cloning.txt.merge',
 '1402_cloning.txt.merge',
 '24_cloning.txt.merge',
 '1288_cloning.txt.merge',
 '2200_cloning.txt.merge',
 '575_cloning.txt.merge',
 '414_cloning.txt.merge',
 '2214_cloning.txt.merge',
 '517_cloning.txt.merge',
 '327_cloning.txt.merge',
 '2497_cloning.txt.merge',
 '1977_cloning.txt.merge',
 '1562_cloning.txt.merge',
 '84_cloning.txt.merge',
 '2424_cloning.txt.merge',
 '30_cloning.txt.merge',
 '1151_cloning.txt.merge',
 '2024_cloning.txt.merge',
 '1599_cloning.txt.merge',
 '518_cloning.txt.merge',
 '130_cloning.txt.merge',
 '541_cloning.txt.merge',
 '832_cloning.txt.merge',
 '704_cloning.txt.merge',
 '289_cloning.txt.merge',
 '190_cloning.txt.merge',
 '1563_cloning.txt.merge',
 '361_cloning.txt.merge',
 '87_cloning.txt.merge',
 '534_cloning.txt.merge',
 '26_cloning.txt.merge',
 '151_cloning.txt.merge',
 '479_cloning.txt.merge',
 '2973

In [8]:
#Delete the " symbols from texts

for i in range(len(merge_list)):#range(int(len(merge_list)*0.8)):
  merge = "/content/" + name_of_task + "_merge/" + merge_list[i]
  with open(merge, "r") as fin:
    old_data = fin.read()
    #print(old_data)
    
  new_data = re.sub("\"", "", old_data)
  #print(new_data)

  with open(merge, "w") as fin:
    fin.write(new_data)
    fin.close()

In [ ]:
# for Amazon dataset
name_of_task = 'rev1'
path = '/content/' + name_of_task + '_merge'
d = {'words': [], 'class': []}
rez = sorted(os.listdir(path))
for n, item in enumerate(rez):
    print (item)
    df = pd.read_csv(path + "/" + item, sep = '\t', header = None)
    d['words'].append(' '.join(df.dropna()[2]))
    d['class'].append(1)

Pre-processing functions

In [9]:
def del_brackets(text):
    return re.sub(r'[\'*\)*\(*]', '', text)

In [10]:
def find_left_head(terminal_nodes_dict, row):
  k = row.start
  while (terminal_nodes_dict[k] != 'Nucleus' and k < row.end):
    k += 1
  return k

In [11]:
def process_bracket(path, file_name, out_file):
  result = os.stat(os.path.join(path, file_name))
  ddg = {}
  if (result.st_size != 0): 
    br = pd.read_csv(os.path.join(path, file_name), sep=', ', header=None, names = ['start', 'end', 'type', 'RST'])
    #print(br)
    for j in br.columns:
      br[j] = br[j].apply(del_brackets)
    br = br.astype({'start': 'int64', 'end': 'int64', 'RST': 'string', 'type': 'string'})
    terminal_nodes = br[br.start == br.end]
    terminal_nodes_dict = {}
    for index, row in terminal_nodes.iterrows():
      terminal_nodes_dict[row.start] = row.type
    n_s = br[br['RST'] == 'span']
    sat = br[br['RST'] != 'span']

    # Dependency discourse graph
    
    max_rel_number = len(terminal_nodes_dict)
    #print(max_rel_number)
  
    ind = 0
    for index, row in n_s.iterrows():
      if (row.start == row.end):
        #print("==", row.start)
        candidate = sat[(sat.start == row.start + 1) & (sat.type == 'Satellite')]
        if (row.start == 1 and candidate.shape[0] > 1):
          candidate = sat[(sat.start == row.start + 1) & (sat.type == 'Satellite') & ((sat.end == max_rel_number) | (sat.end != sat.start))]
          #if (file_name == '1097_marijuana_legalization.txt.brackets'):
          #  print(candidate)
        #print(candidate)
        if (candidate.shape[0] == 0):
          candidate = sat[(sat.end == row.start - 1) & (sat.type == 'Satellite')]
          #if (file_name == '1097_marijuana_legalization.txt.brackets'):
          #    print(sat)
          #    print(row.start)
          #    print(row.end)
              #print(ddg)
              #print(max_rel_number)
              #print("<<", ind)
              #print(row.start, row.end)
              #print(ddg)
          if (candidate.start.values[0] != candidate.end.values[0]):
            head_start = find_left_head(terminal_nodes_dict, candidate.iloc[0])
            ddg[ind] = [head_start, row.start, candidate.RST.values[0]]
          else:
            ddg[ind] = [candidate.start.values[0], row.start, candidate.RST.values[0]]
        else:
          ddg[ind] = [row.start, candidate.start.values[0], candidate.RST.values[0]]
        ind += 1
      elif (row.start < row.end and row.end == max_rel_number):
        #print("<=", ind)
        #print(row)
        candidate = sat[(sat.end == row.start - 1) & (sat.type == 'Satellite')]
        ddg[ind] = [candidate.start.values[0], row.start, candidate.RST.values[0]]
        ind += 1
      elif (row.start < row.end and row.end < max_rel_number):
        #  print("<<", ind)
        #  print(row.start, row.end)
        try:
          candidate = sat[(sat.start == row.end + 1) & (sat.type == 'Satellite')]
          head_start = find_left_head(terminal_nodes_dict, row)
          ddg[ind] = [head_start, candidate.start.values[0], candidate.RST.values[0]]
          ind += 1
        except IndexError:
          candidate = sat[(sat.end == row.start - 1) & (sat.type == 'Satellite')]
          ddg[ind] = [candidate.start.values[0], row.start, candidate.RST.values[0]]
          ind += 1
        
        #if (file_name == '2687_school_uniforms.txt.brackets'):
        #  print(candidate.start.values[0])
        #  print(candidate.RST.values[0])
        #  print(ddg)
        #  print(max_rel_number)
        #  print("<<", ind)
        #  print(row.start, row.end)
        #  print(ddg)

    sat_nucl = sat[sat.type == 'Nucleus']
    sat_nucl.sort_values(by='RST', inplace = True)
    #print('sat_nucl', sat_nucl)
    for i in range(0, sat_nucl.shape[0], 2):
      #print("list", i)
      row = sat_nucl.iloc[i]
      head_start = find_left_head(terminal_nodes_dict, row)
      candidate = sat_nucl.iloc[i+1]
      head_end = find_left_head(terminal_nodes_dict, candidate)
      ddg[ind] = [min(head_start, head_end), max(head_start, head_end), candidate.RST]
      ind += 1

    #print(ddg)
    DDG = pd.DataFrame.from_dict(ddg, orient = 'index')
    #print(DDG)
    DDG.to_csv(out_file + file_name[0:-9], sep = '\t', header = None, index = None)
    return DDG
  else:
    ddg[0] = [1, 1, 'none']
    DDG = pd.DataFrame.from_dict(ddg, orient = 'index')
    DDG.to_csv(out_file + file_name[0:-9], sep = '\t', header = None, index = None)
    return DDG

In [13]:
#Define topic for the UKP corpus
name_of_task = "cloning"

In [14]:
#create the folder for the DDG for some UKP dataset
os.mkdir("ddg_" + name_of_task)

In [15]:
#Creating DDG
path = '/content/' + name_of_task + '_brackets'
rez = sorted(os.listdir(path))
for n, item in enumerate(rez):
    print (item)
    process_bracket(path, item, '/content/ddg_'+ name_of_task + '/')

0_cloning.txt.brackets
1002_cloning.txt.brackets
1003_cloning.txt.brackets
100_cloning.txt.brackets
1010_cloning.txt.brackets
1018_cloning.txt.brackets
1021_cloning.txt.brackets
1027_cloning.txt.brackets
1028_cloning.txt.brackets
102_cloning.txt.brackets
103_cloning.txt.brackets
1042_cloning.txt.brackets
104_cloning.txt.brackets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarnin

1052_cloning.txt.brackets
1059_cloning.txt.brackets
105_cloning.txt.brackets
1064_cloning.txt.brackets
1065_cloning.txt.brackets
1069_cloning.txt.brackets
106_cloning.txt.brackets
1075_cloning.txt.brackets
1076_cloning.txt.brackets
1078_cloning.txt.brackets
1080_cloning.txt.brackets
1082_cloning.txt.brackets
1085_cloning.txt.brackets
1089_cloning.txt.brackets
108_cloning.txt.brackets
1090_cloning.txt.brackets
1092_cloning.txt.brackets
1095_cloning.txt.brackets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 

1096_cloning.txt.brackets
109_cloning.txt.brackets
10_cloning.txt.brackets
1102_cloning.txt.brackets
1103_cloning.txt.brackets
1109_cloning.txt.brackets
110_cloning.txt.brackets
1110_cloning.txt.brackets
1115_cloning.txt.brackets
1117_cloning.txt.brackets
1119_cloning.txt.brackets
1122_cloning.txt.brackets
112_cloning.txt.brackets
1130_cloning.txt.brackets
1132_cloning.txt.brackets


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 

891_cloning.txt.brackets
8_cloning.txt.brackets
901_cloning.txt.brackets
909_cloning.txt.brackets
90_cloning.txt.brackets
913_cloning.txt.brackets
914_cloning.txt.brackets
916_cloning.txt.brackets
918_cloning.txt.brackets
920_cloning.txt.brackets
928_cloning.txt.brackets
92_cloning.txt.brackets
931_cloning.txt.brackets
939_cloning.txt.brackets
943_cloning.txt.brackets
94_cloning.txt.brackets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 

95_cloning.txt.brackets
962_cloning.txt.brackets
965_cloning.txt.brackets
966_cloning.txt.brackets
967_cloning.txt.brackets
96_cloning.txt.brackets
974_cloning.txt.brackets
976_cloning.txt.brackets
97_cloning.txt.brackets
981_cloning.txt.brackets
984_cloning.txt.brackets
987_cloning.txt.brackets
989_cloning.txt.brackets
98_cloning.txt.brackets
994_cloning.txt.brackets
99_cloning.txt.brackets
9_cloning.txt.brackets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: SettingWithCopyWarnin

In [16]:
import os
import zipfile

fantasy_zip = zipfile.ZipFile('ddg_' + name_of_task + '.zip', 'w')
 
for folder, subfolders, files in os.walk('/content/ddg_' + name_of_task):
 
    for file in files:
        if file.endswith('.txt'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '/content'), compress_type = zipfile.ZIP_DEFLATED)
fantasy_zip.close()

## Processing DDG files with the merge files

In [17]:
merge_list = os.listdir(path="/content/" + name_of_task + "_merge")

In [18]:
import re

In [19]:
#for UKP only
distr = pd.read_csv(name_of_task + "_distr.csv")
distr.head()
if ('_' in name_of_task):
  distr.files = distr.files.apply(lambda x: re.sub(r'[a-z_]*/', r'', x))
else:# (name_of_task == 'abortion' or name_of_task == 'cloning'):
  distr.files = distr.files.apply(lambda x: re.sub(r'[a-z]*/', r'', x))

In [20]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def arraytostring(ar):
  marks = '''!()-[]{};?@#$%:'"\,./^&;*_'''
  s = ""
  for a in ar:
    #print(a)
    s += a.lower() + ' '
      
  #print(s) 
  for x in s:  
    if x in marks:  
        s = s.replace(x, "")
  
  return s[:len(s) - 1]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
def arraytostring(ar):
  s = ""
  for a in ar:
    s += str(a) + ' '
  return s[:len(s) - 1]

In [ ]:
#for amazon reviews
def create_df_amazon(merge_list, ukp_topic, class_of_argumentation):
  '''
    Organize the structure of the future dataframe extended with the rhetoric relations
    'words' are the tokens from the merge file corresponding to the specific EDU
    'rst' is the type of rhetoric relatio
    'class' defines the argumentation class
    'segment' defines to which of the text the pair of EDUs relate to
  '''

  data = {'words': [], 'rst' : [], 'class': [], 'segment': []}
  df = pd.DataFrame.from_dict(data)
  #df_out = pd.DataFrame.from_dict(data)

  for i in range(len(merge_list)):#range(int(len(merge_list)*0.8)):
      
      #print(i)
    try:
      merge = "/content/" + ukp_topic + "_merge/" + merge_list[i]
      ddg = "/content/ddg_" + ukp_topic + "/" + merge_list[i][0:-6]
      #print(merge)
      
      data_merge = pd.read_table(merge, header = None)
      data_distr = pd.read_table(ddg, header = None)

      #print(merge_list[i][0:-6])
      # all the words in the data_merge (in one text)
      # review = data_merge[2].values
      #print(review)

      #print(data_distr.shape[0])
      for j in range(data_distr.shape[0]):
        #print(review[data_merge[data_merge[9].isin(range(data_distr.iloc[j][0], data_distr.iloc[j][1] + 1))].index.values])
        s = arraytostring(data_merge[data_merge[9] == data_distr[0][j]][2].values)
        s += ' ' + arraytostring(data_merge[data_merge[9] == data_distr[1][j]][2].values)
        #print(s)
        if (len(s) > 0):
          data['words'].append(s)
          data['rst'].append(data_distr.iloc[j][2])
          segm = re.search(r'_\d{1,7}', merge_list[i])
          data['segment'].append(merge_list[i][segm.start() + 1: segm.end()])
          arg_class = class_of_argumentation
          data['class'].append(arg_class)
        else:
          continue
      df = pd.DataFrame.from_dict(data)
      #else:
      #  df_add = pd.DataFrame.from_dict(data)
      #  df = df.append(df_add, ignore_index = True)
      #print(df.shape)
      
    except Exception:
      print("Exception", i)
      continue
    #print(data)
    #  df_add = pd.DataFrame.from_dict(data)
    #  df = df.append(df_add, ignore_index = True)
  #df = pd.DataFrame.from_dict(data)
  return df

In [ ]:
df = create_df_amazon(merge_list, name_of_task, 'Argument')

In [22]:
#for UKP
def create_df(merge_list, ukp_topic):
  '''
    Organize the structure of the future dataframe extended with the rhetoric relations
    'words' are the tokens from the merge file corresponding to the specific EDU
    'rst' is the type of rhetoric relatio
    'class' defines the argumentation class
    'segment' defines to which of the text the pair of EDUs relate to
  '''

  data = {'words': [], 'rst' : [], 'class': [], 'segment': []}
  df = pd.DataFrame.from_dict(data)
  #df_out = pd.DataFrame.from_dict(data)

  for i in range(len(merge_list)):#range(int(len(merge_list)*0.8)):
      
      #print(i)
    #try:
      merge = "/content/" + ukp_topic + "_merge/" + merge_list[i]
      ddg = "/content/ddg_" + ukp_topic + "/" + merge_list[i][0:-6]
      #if i == 386:
      #  print(merge, ddg)
      
      data_merge = pd.read_table(merge, header = None)
      data_distr = pd.read_table(ddg, header = None)

      # all the words in the data_merge (in one text)
      # review = data_merge[2].values
      #print(review)

      #print(data_distr.shape[0])
      for j in range(data_distr.shape[0]):
        #print(review[data_merge[data_merge[9].isin(range(data_distr.iloc[j][0], data_distr.iloc[j][1] + 1))].index.values])
        s = arraytostring(data_merge[data_merge[9] == data_distr[0][j]][2].values)
        s += ' ' + arraytostring(data_merge[data_merge[9] == data_distr[1][j]][2].values)
        #print(s)
        if (len(s) > 0):
          data['words'].append(s)
          data['rst'].append(data_distr.iloc[j][2])
          data['segment'].append(int(re.search(r'\d{1,6}', merge_list[i]).group(0)))
          arg_class = distr[distr.files == merge_list[i][0:-6]].annotation.iloc[0]
          if (arg_class == 'Argument_for' or arg_class == 'Argument_against'):
            data['class'].append('Argument')
          else:
            data['class'].append(arg_class)
        else:
          continue
      #if (i == 0):
      df = pd.DataFrame.from_dict(data)
      #else:
      #  df_add = pd.DataFrame.from_dict(data)
      #  df = df.append(df_add, ignore_index = True)
      #print(df.shape)
      
    #except Exception:
    #  print("Exception", i)
    #  continue
    #print(data)
    #  df_add = pd.DataFrame.from_dict(data)
    #  df = df.append(df_add, ignore_index = True)
  #df = pd.DataFrame.from_dict(data)
  return df

In [23]:
df = create_df(merge_list, name_of_task)

In [24]:
segm = df.groupby(['segment']).groups
segm

{0: [969, 970, 971], 1: [946], 2: [123], 3: [829], 4: [1388, 1389], 5: [1392], 6: [521], 7: [1501], 8: [779], 9: [1089], 10: [1138, 1139], 11: [1629], 13: [1088], 14: [1578, 1579, 1580], 15: [190], 16: [1007], 17: [337, 338], 18: [1542, 1543], 19: [219, 220], 20: [325], 21: [1120, 1121, 1122, 1123, 1124], 22: [967], 23: [1565], 24: [20], 25: [650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666], 26: [76], 27: [1605, 1606, 1607], 28: [1431, 1432, 1433], 29: [393, 394], 30: [41], 31: [1398, 1399], 32: [1302, 1303, 1304, 1305, 1306], 33: [1681, 1682, 1683], 34: [1020], 35: [1528], 36: [554], 37: [473, 474], 38: [1047], 39: [12, 13, 14, 15], 40: [806], 41: [1488, 1489], 42: [1671], 43: [475, 476, 477], 44: [602], 45: [774, 775, 776, 777], 47: [298, 299, 300, 301, 302, 303, 304, 305, 306], 48: [1420], 49: [162], 50: [943, 944, 945], 51: [1051], 52: [918], 53: [849, 850, 851, 852], 54: [1600, 1601, 1602], 57: [1248], 58: [881], 59: [1371], 60: [378, 379], 62: 

In [ ]:
#for Amazon
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df.rst.values)
df['rst_code'] = le.transform(df.rst.values)
le.classes_

In [ ]:
df['class_code'] = 1

In [ ]:
arg0_df = df[['words', 'rst', 'class', 'segment', 'class_code']]
arg0_df.to_csv("arg0_rev.tsv", sep='\t')

In [ ]:
df.to_csv("arg1_rev.tsv", sep='\t')

In [ ]:
amazon_df = df.append(arg0_df)

In [ ]:
segm = amazon_df.groupby(['segment']).groups

In [ ]:
amazon_df

In [ ]:
#for Amazon
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(amazon_df.rst.values)
amazon_df['rst_code'] = le.transform(amazon_df.rst.values)
le.classes_

In [ ]:
amazon_df.groupby('segment').count()

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test = train_test_split(list(segm.keys()), test_size=0.33, shuffle=True)

In [27]:
train = df[df.segment.isin(X_train)]
train.head()

,words,rst,class,segment
4,and btw if the mother had a clone,condition,Argument,987
5,the father would keep in mind,elaboration,Argument,987
6,would keep in mind that its wrong to do things,elaboration,Argument,987
7,that its wrong to do things to his daughter (,attribution,Argument,987
8,to his daughter ( even if its a clone ) THe cl...,attribution,Argument,987


In [29]:
X_train, X_test = train_test_split(train, test_size=0.33, shuffle=True)